In [1]:
import pandas as pd

courses = pd.read_csv('courses.csv')

In [2]:
concepts = """
#
# CONCEPTS
#

concept: ~my_negation (~negation "don't")
concept: ~greater (greater bigger)
concept: ~lesser (lesser smaller)
concept: ~equal (equal same equivalent equally identical)
concept: ~comparison (~greater ~lesser ~equal)
concept: ~finish (finish complete pass)
concept: ~have (have collect earn complete gather)

# ATTRIBUTES
concept: ~ects (ects ECTS point "ects point" "ECTS point" "ects points" "ECTS points")
concept: ~course (course subject lecture class exercise laboratory)
concept: ~term (term semester "winter semester" "summer semester" "winter term" "summer term" winter summer)
concept: ~teacher (tutor teacher "academic teacher" practitioner professor trainer)
concept: ~classtype (lecture class lab exercise project type)

# COURSES  # TODO
concept: ~course_name (""" \
    + \
    ' '.join(courses['subject'].drop_duplicates().apply(lambda x: '"' + x.lower().replace('\n', '').replace('\r', '').replace('#', '').replace('*', '') + '"').values.tolist()) \
    + \
""")

# NUMBERS
concept: ~one    (one    "1"  1 1st first)
concept: ~two    (two    "2"  2 2nd second)
concept: ~three  (three  "3"  3 3rd third)
concept: ~four   (four   "4"  4 4th fourth)
concept: ~five   (five   "5"  5 5th fifth)
concept: ~six    (six    "6"  6 6th sixth)
concept: ~seven  (seven  "7"  7 7th seventh)
concept: ~eight  (eight  "8"  8)
concept: ~nine   (nine   "9"  9)
concept: ~ten    (ten    "10" 10)
concept: ~eleven (eleven "11" 11)
concept: ~twelve (twelve "12" 12)

concept: ~number (~one ~two ~three ~four ~five ~six ~seven ~eight ~nine ~ten ~eleven ~twelve)

"""
print(concepts)  # TODO: sprawdż czy działa ze znakiem # w cudzysłowie


#
# CONCEPTS
#

concept: ~my_negation (~negation "don't")
concept: ~greater (greater bigger)
concept: ~lesser (lesser smaller)
concept: ~equal (equal same equivalent equally identical)
concept: ~comparison (~greater ~lesser ~equal)
concept: ~finish (finish complete pass)
concept: ~have (have collect earn complete gather)

# ATTRIBUTES
concept: ~ects (ects ECTS point "ects point" "ECTS point" "ects points" "ECTS points")
concept: ~course (course subject lecture class exercise laboratory)
concept: ~term (term semester "winter semester" "summer semester" "winter term" "summer term" winter summer)
concept: ~teacher (tutor teacher "academic teacher" practitioner professor trainer)
concept: ~classtype (lecture class lab exercise project type)

# COURSES  # TODO
concept: ~course_name ("project: coreforge" "financial services" "course: game development in unreal engine 5" "combinatorial optimization under uncertainty" "tutoring" "combinatorial optimization" "digital image processing" "algebra

In [3]:
def process_line(line):
    return " ".join(['"' + str(term).lower().replace('\n', '').replace('\r', '').replace('#', '').replace('*', '').strip() + '"' for term in line[:-1]]) + " * "

In [4]:
data = """
#
# DATA
#

# BASIC COURSE INFO
table: ~subjectects(^subject ^ects ^classtype ^term ^offered ^firstyear ^maintutor)

if (^ects != *) { createfact(^subject ects ^ects) }
if (^classtype != *) { createfact(^subject classtype ^classtype) }
if (^term != *) { createfact(^subject term ^term) }
if (^offered != *) { createfact(^subject offer ^offered) }

DATA:
""" \
    + \
    "\n".join([process_line(l) for l in courses.values.tolist()])

data

'\n#\n# DATA\n#\n\n# BASIC COURSE INFO\ntable: ~subjectects(^subject ^ects ^classtype ^term ^offered ^firstyear ^maintutor)\n\nif (^ects != *) { createfact(^subject ects ^ects) }\nif (^classtype != *) { createfact(^subject classtype ^classtype) }\nif (^term != *) { createfact(^subject term ^term) }\nif (^offered != *) { createfact(^subject offer ^offered) }\n\nDATA:\n"project: coreforge" "4" "analiza kodu źródłowego, prezentacja, projekt programistyczny" "winter" "true" "false" * \n"financial services" "5" "wykład, ćwiczenia" "winter" "true" "false" * \n"course: game development in unreal engine 5" "5" "wykład, indywidualny lub zespołowy projekt programistyczny, samodzielna praca przy komputerze" "winter" "true" "false" * \n"combinatorial optimization under uncertainty" "6" "wykład" "winter" "true" "false" * \n"tutoring" "0" "nan" "winter" "true" "true" * \n"combinatorial optimization" "6" "ćwiczenia audytoryjne wykład monograficzny" "winter" "true" "false" * \n"digital image processin

In [5]:
topic_intro = """
#
#
# TOPICS
#
#
"""

In [13]:
def course_question(line):
    try:
        course_name = '"' + line[0].lower() + '"'
        all = line[-1].replace("#", "").replace("*", "").replace("\"", "").split("\n") if "\n" in line[-1] else [line[-1], line[-1]]
        basic = "{\"" + " ".join(all[:2]).replace('{', '').replace('}', '').replace('\r', ' ').replace('\n', ' ').replace("_", " ").replace("(", "").replace("[", "").replace(")", "").replace("]", "").replace("/", " ").strip()  + " \"}"
        detailed = "{\"" + " ".join(all[2:12]).replace('{', '').replace('}', '').replace('\r', '').replace('\n', ' ').replace("_", " ").replace("(", "").replace("[", "").replace(")", "").replace("]", "").replace("/", " ").strip() + " \"}"
        if detailed == "{}":
            detailed = basic
        return f"""
u: ([about happen] * {course_name}) ^repeat() ^keep()
    {basic}
    a: (!~my_negation * more)
        {detailed}
"""
    except:
        return f"""
u: ([about happen] * {line[0].lower()}) ^repeat() ^keep()
    Sorry, there was no information for {line[0].lower()} provided."""

In [14]:
topic_course_info = """
#
# COURSE INFO
#

topic: ~course_info (~course_name)

t: $finished_classes = ^jsoncreate(PERMANENT array)
Hello, talk to me!

t: Do you want to learn about the offer of CS institute? ^keep()
	a: ([~yes yes]) Which of the subjects are you interested in? Maybe $cur=^pick(~course_name) $cur?
		b: (~ects) ^reuse( CHECKECTS )
		b: ([when ~term]) ^reuse( CHECKTERM )
		b: (~classtype) ^reuse ( CHECKTYPE )
		b: (offer) ^reuse( CHECKOFFER )
		b: (~finish) ^reuse( COURSEFINISHED )

u: COMPULSORYCOURSES (compulsory !~course_info)
	The compulsory courses are: """ \
    + \
    'AN, MD, LdI...' \
    + \
"""

u: COURSEFINISHED (~finish {_~course_name}) ^repeat() ^keep()
    $temp = ^join( $cur '_0 )
    if ($cur != $temp) { $cur = '_0' }
    ^jsonarrayinsert( UNIQUE $finished_classes $cur )
    $_formatted_classes = ^jsonwrite( $finished_classes )
    Okay, so you've already finished this course. Your finished classes are: $_formatted_classes.

u: CHECKECTS ([many ~number] ~ects <<get>> {_~course_name}) ^repeat() ^keep()
    $temp = ^join( $cur '_0 )
    if ($cur != $temp) { $cur = '_0' }
  	if (^query(direct_sv $cur ects ?)) { You can get @0object ECTS points for the @0subject. }
  	else {I have no idea about $cur}

u: CHECKTERM ([when ~term] < * {_~course_name}) ^repeat() ^keep()
    $temp = ^join( $cur '_0 )
    if ($cur != $temp) { $cur = '_0' }
  	if (^query(direct_sv $cur term ?)) { The course @0subject is offered in the @0object term. }
  	else {I have no idea about $cur}

  	a: ([now "this year" "this time"]) ^reuse ( CHECKOFFER )

u: CHECKTYPE (~classtype < * {_~course_name}) ^repeat() ^keep()
    $temp = ^join( $cur '_0 )
    if ($cur != $temp) { $cur = '_0' }
  	if (^query(direct_sv $cur classtype ?)) { The course @0subject is offered in the @0object form. }
  	else {I have no idea about $cur}

u: CHECKOFFER (offer < * {_~course_name}) ^repeat() ^keep()
    $temp = ^join( $cur '_0 )
    if ($cur != $temp) { $cur = '_0' }
	if (^query(direct_sv $cur offer ?))  {
		if (@0object == "true") { The course @0subject is offered in the current cycle. }
		else { The course @0subject is not offered in the current cycle. Please try next year. } }
  	else {I have no idea about $cur}

  	a: (~term) ^reuse ( CHECKTERM )

u: REQUIREMENTS ([require requirement] < * {_~course_info}) ^repeat() ^keep()
    $temp = ^join( $cur '_0 )
    if ($cur != $temp) { $cur = '_0' }
	I don't know the requirements for $cur.

u: CHECKFIRSTYEAR (first < * appropriate < * {_~course_name}) ^repeat() ^keep()
    $temp = ^join( $cur '_0 )
    if ($cur != $temp) { $cur = '_0' }
	if (^query(direct_sv $cur firstyear ?))  {
		if (@0object == "true") { The course @0subject is appropriate for the first year students. }
		else { The course @0subject is not appropriate for the first year students. Please try next year. } }
  	else {I have no idea about $cur}

#! How many ects I have now?
?: ECTSCOUNTER ([many much] ~ects * I ~have *) ^repeat() ^keep()
    $_counter = 0
    $_points = 0
    $_len = ^length($finished_classes)
    ^loop($_len)
    {
        if (^query(direct_sv $finished_classes[$_counter] ects ?)) {$_points += @0object}
        $_counter += 1
    }
    $_formatted_classes = ^jsonwrite( $finished_classes )
    You have $_points ECTS points, your finished classes are: $_formatted_classes.

""" \
    + \
    "\n\n".join([course_question(l) for l in courses[['subject', 'description']].drop_duplicates().values.tolist()])


In [15]:
with open("RAWDATA/CriStian/bots.top", "w") as f:
    f.write((concepts + data + topic_intro + topic_course_info).replace('"nan"', '0').replace("—", "-").replace("–", "-"))

In [ ]:
print(concepts + data + topic_intro + topic_course_info)

In [ ]:
line[-1]

In [ ]:
"–" == "-"